# Public Prize Draft

In [1]:
import pandas as pd
import numpy as np

pd.options.display.max_rows = 1_000

### Pool Settings

In [2]:
# Yahoo Public Leagues

TEAMS = 12
ROSTER = {'C': 2, 'LW': 2, 'RW': 2, 'D': 4, 'G': 2}

SKATER_CATEGORIES = {
    'goals': 7/8, 
    'assists': 1, 
    'plus_minus': 3/4, 
    'powerplay_points': 3/4, 
    'shots_on_goal': 1, 
    'hits': 1
}

GOALIE_CATEGORIES = {
    "wins" : 1, 
    "goals_against_average": 7/8,
    "save_percentage": 7/8, 
    "shutouts": 1/2
}

### Yahoo Ranking Data

In [3]:
age = pd.read_csv("../data/info-capfriendly_com.csv")
age = age[["name", "age"]]

yahoo = pd.read_csv("../data/projections-yahoo_com.csv")
yahoo["yahoo_rank"] = (yahoo["ranking_preseason"] + yahoo["ranking_current"]) / 2
yahoo = pd.merge(yahoo, age, how="left", on="name")

yahoo = yahoo[["team", "age", "name", "position", "yahoo_rank"]]

### Multiple Positions

In [4]:
multi = (
    yahoo
    [["name", "position"]]
    .set_index(['name']) 
    .apply(lambda col: col.str.split(',').explode())
    .reset_index()
)

multi.head()

,name,position
0,Nathan MacKinnon,C
1,Alex Ovechkin,LW
2,Connor McDavid,C
3,Artemi Panarin,LW
4,Leon Draisaitl,C


### Projection Data

In [5]:
proj = pd.read_csv("../data/projections-generated.csv")

### Goalies

In [6]:
goalies = proj[proj["position"] == "G"].copy()

goalies = (
    goalies
    [["name"] + list(GOALIE_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)

for key, value in GOALIE_CATEGORIES.items():
    goalies[key] *= value
    
goalies["rollup"] = goalies.apply(lambda row: row.sum(), axis=1)
goalies["rollup"] /= sum(GOALIE_CATEGORIES.values())
goalies["rollup"] *= 100

goalies = goalies.reset_index()
goalies = goalies[["name", "rollup"]]

goalies.head()

,name,rollup
0,Andrei Vasilevskiy,95.315705
1,Connor Hellebuyck,69.446289
2,Tuukka Rask,71.534501
3,Carter Hart,66.108638
4,Robin Lehner,74.598128


### Skaters

In [7]:
skaters = proj[proj["position"] != "G"].copy()
skaters = (
    skaters
    [["name"] + list(SKATER_CATEGORIES.keys())]
    .set_index("name")
    .apply(lambda x: (x - x.min()) / (x.max() - x.min()))
)
     
for key, value in SKATER_CATEGORIES.items():
    skaters[key] *= value
    
skaters["rollup"] = skaters.apply(lambda row: row.sum(), axis=1)
skaters["rollup"] /= sum(SKATER_CATEGORIES.values())
skaters["rollup"] *= 100

skaters = skaters.reset_index()
skaters = skaters[["name", "rollup"]]

skaters.head()

,name,rollup
0,Connor McDavid,69.556884
1,Nathan MacKinnon,74.087378
2,Leon Draisaitl,65.928546
3,Artemi Panarin,63.300636
4,Alex Ovechkin,67.336683


### Merge

In [8]:
rollup = pd.concat([skaters, goalies])
rollup = pd.merge(multi, rollup, how="inner", on="name").sort_values("rollup", ascending=False)
rollup.head()

,name,position,rollup
284,Andrei Vasilevskiy,G,95.315705
293,Anton Khudobin,G,75.885960
287,Robin Lehner,G,74.598128
0,Nathan MacKinnon,C,74.087378
307,Philipp Grubauer,G,73.053458


### VORP

In [9]:
vorp = rollup.copy()
vorp["vorp"] = vorp["rollup"]

for position, slots in ROSTER.items():
    replacement = (
        vorp[vorp['position'] == position]
        .sort_values('vorp', ascending=False)
        .head(slots * TEAMS)
        ['vorp']
        .mean()
    )
    vorp.loc[vorp['position'] == position, 'vorp'] = vorp['vorp'] - replacement

vorp.head()

,name,position,rollup,vorp
284,Andrei Vasilevskiy,G,95.315705,32.998451
293,Anton Khudobin,G,75.885960,13.568706
287,Robin Lehner,G,74.598128,12.280874
0,Nathan MacKinnon,C,74.087378,18.992840
307,Philipp Grubauer,G,73.053458,10.736204


### Lineups

In [20]:
lineups = pd.read_csv("../data/lineups-dailyfaceoff_com.csv")
lineups["name"] = lineups["name"].replace("Nathan Mackinnon", "Nathan MacKinnon")

### Draft

In [21]:
draft = pd.merge(yahoo, vorp, how="left", on="name", suffixes=("_yahoo", ""))
draft = pd.merge(draft, lineups, how="left", on="name")

In [24]:
draft = draft[[
    'team', 'age', 'name', 
    'line', "ppu",
    'position_yahoo', 'position',
    'rollup', 'vorp', 'yahoo_rank',
]]

draft["rollup"] = draft["rollup"].round(1)
draft["vorp"] = draft["vorp"].round(1)
draft['rp'] = draft.groupby("position")['yahoo_rank'].rank(ascending=True)
draft['rp_vorp'] = draft.groupby("position")['vorp'].rank(ascending=False)
draft["rp_arbitrage"] =  draft['rp'] - draft['rp_vorp']
draft['round'] = draft["yahoo_rank"] // TEAMS + 1
draft = draft.sort_values("yahoo_rank", ascending=True)

In [26]:
pd.DataFrame(draft.groupby(["round"])['position'].value_counts())

position
round position          
1.0   C                7
      LW               4
      G                1
      RW               1
2.0   RW               5
      LW               4
      C                2
      D                1
      G                1
3.0   LW               5
      C                4
      RW               4
      D                2
      G                2
4.0   G                4
      LW               4
      RW               4
      C                2
      D                1
5.0   C                5
      D                3
      G                2
      LW               1
      RW               1
6.0   LW               5
      C                4
      RW               4
      D                1
      G                1
7.0   C                3
      D                3
      G                3
      LW               3
      RW               1
8.0   C                7
      LW               6
      RW               6
      G                2
9.0   D                5
      RW               3
      C                1
      G                1
      LW               1
10.0  LW               6
      RW               4
      C                3
      D                2
      G                1
11.0  LW               3
      RW               3
      C                2
      D                2
      G                2
12.0  RW               8
      LW               7
      D                4
      G                3
      C                1
13.0  RW               4
      C                3
      D                2
      LW               2
14.0  D                6
      LW               2
      RW               2
15.0  RW               7
      D                6
      LW               4
      C                3
      G                2
16.0  LW               3
      RW               3
      D                2
      G                2
      C                1
17.0  G                4
      RW               4
      C                3
      LW               3
      D                1
18.0  C                5
      D                2
      LW               2
      RW               2
19.0  C                5
      RW               4
      G                2
      LW               2
      D                1
20.0  D                3
      LW               2
      RW               2
21.0  D                4
      G                4
      RW               4
      LW               2
22.0  LW               3
      RW               3
      D                1
      G                1
23.0  G                3
      C                1
      D                1
      RW               1
24.0  D                3
      G                1
25.0  D                5
      G                2
      LW               1
26.0  D                2
27.0  G                2
28.0  D                2
      G                2
29.0  G                1
30.0  D                1
32.0  G                1
35.0  C                1
      RW               1
40.0  D                1
44.0  G                1

### Export

In [27]:
draft.to_csv("../data/draft-yahoo_league.csv", index=False)